# BrkRaw Tutorial 2: Orientation, Unwrap Pose vs Subject Pose

This notebook compares scanner-view (unwrap pose) and subject-view
affines for the same dataset. We then reorient both to RAS and visualize
the differences.


## 1. Load the example dataset

This notebook assumes `data/brkraw-dataset` is already available.
If not, run the download step in `00_getting_started.ipynb`.


In [ ]:
from pathlib import Path
import brkraw as brk

DATASET_REPO = Path("data") / "brkraw-dataset"
if not DATASET_REPO.exists():
    raise FileNotFoundError("Dataset repo not found. Run the download step first.")

dataset_zip = DATASET_REPO / "PV6.0.1" / "UNC_PV6.0.1_FLASH_TurboRARE_EPI.zip"
loader = brk.load(str(dataset_zip))


## 2. Helper: reorient to RAS

We will use a simple nibabel-based function to reorient arrays.


In [ ]:
import numpy as np
import nibabel as nib

def reorient_to_ras(data, affine):
    ornt = nib.orientations.io_orientation(affine)
    ras_ornt = np.array([[0, 1], [1, 1], [2, 1]])  # RAS
    transform = nib.orientations.ornt_transform(ornt, ras_ornt)
    new_data = nib.orientations.apply_orientation(data, transform)
    new_affine = affine @ nib.orientations.inv_ornt_aff(transform, data.shape)
    return new_data, new_affine


## 3. Fetch data and affines

We explicitly apply the subject pose (`Head_Supine`) and compare it with
scanner-view affines from `unwrap_pose=True`.


In [ ]:
def first_pack(value):
    return value[0] if isinstance(value, tuple) else value

scan_id = sorted(loader.avail.keys())[0]
reco_id = sorted(loader.avail[scan_id].avail.keys())[0]

data = first_pack(loader.get_dataobj(scan_id, reco_id=reco_id))
affine_subject = first_pack(
    loader.get_affine(scan_id, reco_id=reco_id, override_subject_pose="Head_Supine")
)
affine_scanner = first_pack(
    loader.get_affine(scan_id, reco_id=reco_id, unwrap_pose=True)
)
scan_id, reco_id


## 3b. Override subject type and pose

Because this dataset was labeled as Head_Supine despite being acquired in a Head_Prone setup,
override the subject type and pose to match the true acquisition. For rodents, the subject
type is `Quadruped`.


In [ ]:
affine_override = first_pack(
    loader.get_affine(
        scan_id,
        reco_id=reco_id,
        override_subject_type="Quadruped",
        override_subject_pose="Head_Prone",
    )
)
print("Override (Quadruped, Head_Prone) axcodes:", nib.orientations.aff2axcodes(affine_override))


In [ ]:
print("Subject-view (Head_Supine) axcodes:", nib.orientations.aff2axcodes(affine_subject))
print("Scanner-view (unwrap_pose=True) axcodes:", nib.orientations.aff2axcodes(affine_scanner))


## 4. Reorient to RAS and visualize

We reorient both arrays using their affines, then compare a mid-slice.

Terminology notes:
- **Unwrap pose** means *scanner view* (x, y, z increase toward operator right, top, back).
- **Subject pose applied** means Paravision subject view is wrapped back to the subject base
  orientation using the reported subject pose.

For this dataset: it predates the PV360 preclinical view, and it was recorded with a default
Head_Prone setup while being labeled as Head_Supine for convenience. When the pose is applied
correctly, the image can appear flipped.


In [ ]:
data_subject_ras, affine_subject_ras = reorient_to_ras(data, affine_subject)
data_scanner_ras, affine_scanner_ras = reorient_to_ras(data, affine_scanner)


Note: These are 2D (x, y) axis plots of a single slice. We set `origin="lower"` to match the usual neuroimaging display convention so the image axes align with RAS-style orientation in this 2D view.

In [ ]:
import matplotlib.pyplot as plt

slice_subject = data_subject_ras.shape[2] // 2
slice_scanner = data_scanner_ras.shape[2] // 2

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
axes[0].imshow(np.rot90(data_subject_ras[:, :, slice_subject]), cmap="gray")
axes[0].set_title("Subject pose (Head_Supine)")
axes[1].imshow(np.rot90(data_scanner_ras[:, :, slice_scanner]), cmap="gray")
axes[1].set_title("Unwrap pose (scanner view)")
for ax in axes:
    ax.axis("off")
plt.tight_layout()


## 4b. Scanner-view slice planes (bore coordinates)

These panels are labeled in scanner-bore coordinates: X = right, Y = up, Z = back (operator-facing orientation).

In [ ]:
def plot_orthogonal(data, title):
    i = data.shape[0] // 2
    j = data.shape[1] // 2
    k = data.shape[2] // 2

    views = [
        (np.rot90(data[:, :, k]), "Axial (x, y)"),
        (np.rot90(data[:, j, :]), "Coronal (x, z)"),
        (np.rot90(data[i, :, :]), "Sagittal (y, z)"),
    ]

    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    for ax, (img, label) in zip(axes, views):
        ax.imshow(img, cmap="gray", origin="lower")
        ax.set_title(label)
        ax.axis("off")
    fig.suptitle(title)
    plt.tight_layout()

plot_orthogonal(data_subject_ras, "Subject pose (Head_Supine)")
plot_orthogonal(data_scanner_ras, "Unwrap pose (scanner view)")


## 5. Compare affines

The two affines differ because older datasets report `Head_Supine` and
BrkRaw applies a subject-pose wrap for subject-view outputs.


In [ ]:
np.set_printoptions(precision=3, suppress=True)
affine_subject


In [ ]:
affine_scanner
